In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping

# Loading Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
abs_path = '/content/drive/My Drive/Google_Colab/Crypto_Price_Predictor'

os.chdir(abs_path)

X_train = pickle.load(open('X_train.pickle', 'rb'))
X_test = pickle.load(open('X_test.pickle', 'rb'))
y_train = pickle.load(open('y_train.pickle', 'rb'))
y_test = pickle.load(open('y_test.pickle', 'rb'))

In [ ]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
SEQ_LEN = 60
FUTURE_PRED_PERIOD = 7
COIN_TO_PREDICT = 'ADA'
EPOCHS = 10
BATCH_SIZE = 64
NAME = f'{COIN_TO_PREDICT}-PRED_{SEQ_LEN}-SEQ_{FUTURE_PRED_PERIOD}-PRED_PERIOD'

# Building Model

In [ ]:
model = Sequential()

model.add(LSTM(128, input_shape=(X_train.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(LSTM(128, input_shape=(X_train.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(LSTM(128, input_shape=(X_train.shape[1:])))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(1, activation='sigmoid'))


model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001, decay=1e-6),
              loss='binary_crossentropy', metrics=['accuracy'])

tensorboard = TensorBoard(log_dir=f'logs/{NAME}')
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
model.fit(X_train, y_train, batch_size=BATCH_SIZE,
          epochs=EPOCHS, validation_data=(X_test, y_test),
          callbacks=[tensorboard, early_stopping])

Epoch 1/10
10/10 [==============================] - 0s 41ms/step - loss: 0.4785 - accuracy: 0.7559 - val_loss: 0.7235 - val_accuracy: 0.5000
Epoch 2/10
10/10 [==============================] - 0s 21ms/step - loss: 0.3953 - accuracy: 0.8244 - val_loss: 0.7460 - val_accuracy: 0.5000
Epoch 3/10
10/10 [==============================] - 0s 24ms/step - loss: 0.4078 - accuracy: 0.8094 - val_loss: 0.7724 - val_accuracy: 0.5000
Epoch 4/10
10/10 [==============================] - 0s 20ms/step - loss: 0.3726 - accuracy: 0.8244 - val_loss: 0.8008 - val_accuracy: 0.5000


In [ ]:
val_accruacy = model.evaluate(X_test, y_test)

6/6 [==============================] - 0s 7ms/step - loss: 0.7003 - accuracy: 0.5333


In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred = [1 if x>= 0.5 else 0 for x in y_pred]

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[90  0]
 [84  6]]
              precision    recall  f1-score   support

         0.0       0.52      1.00      0.68        90
         1.0       1.00      0.07      0.12        90

    accuracy                           0.53       180
   macro avg       0.76      0.53      0.40       180
weighted avg       0.76      0.53      0.40       180



# Testing Multiple Models

In [ ]:
layers = [3, 4]
dense = [1, 2, 3]
units = [64, 128, 256]

for layer in layers:
  for dense_layer in dense:
    for unit in units:

      model_name = f'{layer}-LSTM_{dense_layer}-Dense_{unit}-Unit'
      print(model_name)

      model = Sequential()

      model.add(LSTM(unit, input_shape=(X_train.shape[1:]), return_sequences=True))
      model.add(Dropout(0.2))
      model.add(BatchNormalization())
      for i in range(layer-2):
        model.add(LSTM(unit, input_shape=(X_train.shape[1:]), return_sequences=True))
        model.add(Dropout(0.2))
        model.add(BatchNormalization())
      model.add(LSTM(unit, input_shape=(X_train.shape[1:])))
      model.add(Dropout(0.2))
      model.add(BatchNormalization())

      for x in range(dense_layer):
        model.add(Dense(unit, activation='relu'))
        model.add(Dropout(0.2))
        model.add(BatchNormalization())

      model.add(Dense(1, activation='sigmoid'))

      model.compile(optimizer=tf.keras.optimizers.Adam(0.001, decay=1e-6),
                    loss='binary_crossentropy', metrics=['accuracy'])
      
      tensorboard = TensorBoard(log_dir=f'logs/{model_name}')
      filepath = 'RNN-Final-{epoch:02d}'
      checkpoint = ModelCheckpoint('models/{}.model'.format(filepath, monitor='val_acc',
                                                            verbose=1, save_best_only=True, mode='max'))
      
      model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS,
                validation_data=(X_test, y_test),
                callbacks=[tensorboard, early_stopping, checkpoint])

3-LSTM_1-Dense_64-Unit
Epoch 1/10
10/10 [==============================] - 13s 1s/step - loss: 0.7852 - accuracy: 0.5518 - val_loss: 0.6913 - val_accuracy: 0.5611
Epoch 2/10
10/10 [==============================] - 12s 1s/step - loss: 0.6762 - accuracy: 0.6137 - val_loss: 0.6934 - val_accuracy: 0.5000
Epoch 3/10
10/10 [==============================] - 12s 1s/step - loss: 0.5856 - accuracy: 0.6789 - val_loss: 0.6946 - val_accuracy: 0.5000
Epoch 4/10
10/10 [==============================] - 11s 1s/step - loss: 0.5463 - accuracy: 0.7040 - val_loss: 0.6982 - val_accuracy: 0.5000
3-LSTM_1-Dense_128-Unit
Epoch 1/10
10/10 [==============================] - 14s 1s/step - loss: 0.7395 - accuracy: 0.5652 - val_loss: 0.6980 - val_accuracy: 0.5056
Epoch 2/10
10/10 [==============================] - 12s 1s/step - loss: 0.6264 - accuracy: 0.6589 - val_loss: 0.7061 - val_accuracy: 0.5000
Epoch 3/10
10/10 [==============================] - 12s 1s/step - loss: 0.5392 - accuracy: 0.7341 - val_loss: 0.7

After viewing with tensorboard by typing tensorboard --logdir=logs/ in the command line, I determined the best model is the original model we used 3-LSTM, 3-Dense, at 256 units per each layer after 2 epochs.

In [ ]:
final_model = Sequential()
final_model.add(LSTM(256, input_shape=(X_train.shape[1:]), return_sequences=True))
final_model.add(Dropout(0.2))
final_model.add(BatchNormalization())

final_model.add(LSTM(256, input_shape=(X_train.shape[1:]), return_sequences=True))
final_model.add(Dropout(0.2))
final_model.add(BatchNormalization())

final_model.add(LSTM(256, input_shape=(X_train.shape[1:])))
final_model.add(Dropout(0.2))
final_model.add(BatchNormalization())

final_model.add(Dense(256, activation='relu'))
final_model.add(Dropout(0.2))
final_model.add(BatchNormalization())

final_model.add(Dense(256, activation='relu'))
final_model.add(Dropout(0.2))
final_model.add(BatchNormalization())

final_model.add(Dense(256, activation='relu'))
final_model.add(Dropout(0.2))
final_model.add(BatchNormalization())

final_model.add(Dense(1, activation='sigmoid'))

final_model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001, decay=1e-6),
                    loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3)
filepath = 'RNN-Final-{epoch:02d}'
checkpoint = ModelCheckpoint('models/{}.model'.format(filepath, monitor='val_acc',
                                                            verbose=1, save_best_only=True, mode='max'))

final_model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE,
                validation_data=(X_test, y_test), callbacks=[early_stopping, checkpoint])

Epoch 1/10
 9/10 [==========================>...] - ETA: 0s - loss: 0.8290 - accuracy: 0.5191WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/tracking/tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: models/RNN-Final-01.model/assets
10/10 [==============================] - 15s 1s/step - loss: 0.8301 - accuracy: 0.5134 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 2/10
10/10 [==============================] - 14s 1s/step - loss: 0.6868 - accuracy: 0.6505 - val_loss: 0.6890 - val_accuracy: 0.5000
Epoch 3/10
10/10 [==============================] - 14s 1s/step - loss: 0.6734 - accuracy: 0.6472 

In [ ]:
final_model.evaluate(X_test, y_test)

6/6 [==============================] - 0s 10ms/step - loss: 0.6603 - accuracy: 0.6167


[0.6603277921676636, 0.6166666746139526]

In [ ]:
final_model.save('ADA_Crypto_Predictor_Model.h5') 